<a href="https://colab.research.google.com/github/Carine-69/SchoolDropOut-RW/blob/main/SchoolDropOut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
#import dataet
df = pd.read_csv('/content/drive/MyDrive/ML-datasets/SchoolDropOut.csv')

In [5]:
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1.0,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1.0,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1.0,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1.0,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0.0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [7]:
missing_row = df[df.isnull().any(axis=1)]
missing_columns = missing_row.isnull().sum(axis=0)
print('missing columns in that row:')
print(missing_columns[missing_columns > 0])


missing columns in that row:
Daytime/evening attendance\t    1
dtype: int64


In [8]:
df[df['Daytime/evening attendance\t'].isnull()]

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
9,1,1,1,9238,NaN,1,138.0,1,1,19,...,0,6,14,2,13.5,0,8.9,1.4,3.51,Dropout


In [9]:
#clean column name
df.columns = df.columns.str.strip()


In [10]:
#fill the missing value
missing_row = df[df['Daytime/evening attendance'].isnull()]


In [11]:
df.loc[9]

,9
Marital status,1
Application mode,1
Application order,1
Course,9238
Daytime/evening attendance,NaN
Previous qualification,1
Previous qualification (grade),138.0
Nacionality,1
Mother's qualification,1
Father's qualification,19


In [12]:
missing_row = df.loc[9] #target raw
feature = ['Course', 'Application mode', 'Previous qualification', 'Target'] # features to bae on while fill linh=g the target raw
similar_rows_mask = (df['Daytime/evening attendance'].notnull()) & (df[feature]==missing_row[feature]).all(axis=1)
if similar_rows_mask.any():
  df.loc[9, 'Daytime/evening attendance'] = df.loc[similar_rows_mask, 'Daytime/evening attendance'].mode()[0]
  print(df.loc[9])
else:
  print("No similar rows found to impute the missing value.") # find the most similar idataset to match the mising raw and use it's data to fill in the blank and if non found, print 'no similar rows found to impute the missing value'

Marital status                                          1
Application mode                                        1
Application order                                       1
Course                                               9238
Daytime/evening attendance                            1.0
Previous qualification                                  1
Previous qualification (grade)                      138.0
Nacionality                                             1
Mother's qualification                                  1
Father's qualification                                 19
Mother's occupation                                     4
Father's occupation                                     7
Admission grade                                     123.0
Displaced                                               1
Educational special needs                               0
Debtor                                                  1
Tuition fees up to date                                 0
Gender        

In [13]:
df[df.isnull().any(axis=1)]


,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target


In [14]:
#define data role
selected_features = [
    'Course',
    'Application mode',
    'Application order',
    'Previous qualification',
    'Previous qualification (grade)',
    'Admission grade',
    'Age at enrollment',
    'Daytime/evening attendance',
    'Debtor',
    'Tuition fees up to date',
    'Scholarship holder',
    'Educational special needs',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)'
]

x = df[selected_features]
y = df['Target']

#split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# initialize the scaler
scaler = StandardScaler()
# fit and transform the training data
x_train_scaled = scaler.fit_transform(x_train)
# transform the testing data
x_test_scaled = scaler.transform(x_test)

In [15]:
#initialize logistic regression
model = LogisticRegression(max_iter=1000, multi_class='multinomial')
#hperparameter tuning


In [17]:
grid = {
    'C': [1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['saga'],
    'class_weight' :[None,'balanced']
}



In [20]:
# Initialize GridSearchCV
grid_search = GridSearchCV(model, grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(x_train_scaled, y_train)
#best parameters
print("best params", grid_search.best_params_)
print("best score", grid_search.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


best params {'C': 1, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'}
best score 0.7663186536571333


In [24]:
best_model = grid_search.best_estimator_
#make prediction
y_pred = best_model.predict(x_test)
#test the model
cm = confusion_matrix(y_test, y_pred)
test_accuracy = accuracy_score(y_test, y_pred)
print('confusion matrix:')
print(cm)
print('accuracy score:')
print(test_accuracy)

confusion matrix:
[[315   0   1]
 [150   0   1]
 [412   0   6]]
accuracy score:
0.36271186440677966


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
